In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import statsmodels.api as sm
import seaborn as sns
from sklearn import linear_model
import warnings
warnings.simplefilter("ignore")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_excel('/kaggle/input/online-sales/Kaggle Anonymized Data 3.27.12 v4.xlsx') #read in the training data 
df.info()
df.head()

## 1. Pre-processing
### Alter product features and target vairable in scope of the problem statement

In [ ]:
fix = df.iloc[:,0:12].replace(0, np.nan)
fix.head()

In [ ]:
fix['Total Period Sales'] = fix.iloc[:,0:12].sum(axis='columns')
fix['Active Sales Months'] = fix.iloc[:,0:12].count(axis='columns')

In [ ]:
fix = fix.drop(columns = fix.iloc[:,0:12])
fix.head() #dependant vairable and generated time feature

In [ ]:
df = df.drop(columns = df.iloc[:,0:12])
df = fix.join(df)
df.columns = [x.strip() for x in df.columns] #remove spaces in column names

In [ ]:
df['Launch Month'] = df['Date_4'].dt.month #another time feature in independant vairables
df = df.drop(columns = ['Date_4']) #no more use for this given date vairable

In [ ]:
df.head()

### Nan and Missing Columns

In [ ]:
nan_columns = list(df.columns[df.isnull().any()])
missing_columns = list(df.columns[df.isin(['MISSING']).any()])
rouge_columns = nan_columns + missing_columns
rouge_columns[0:10]

In [ ]:
df[rouge_columns] = df[rouge_columns].replace('MISSING', np.nan) #replace NaN values with the median of the column
df[rouge_columns] = df[rouge_columns].replace(np.nan, df.median(axis=0)) #replace NaN values with the median of the column

### Identify True Binary Features
As stated in the beggining, a product categorical feature can either be 0 (it does not possess the categorical feature) or 1 (it does possess the categorical feature). On close inspection of the dataframe above, you will notice there are some categorical measures > 1 or != 0. Upon consultation with the data source in [this thread](https://www.kaggle.com/c/online-sales/discussion/1898) - we can conclude and interpret that any vairable with a categorical feature that is not binary was either measured poorly. There are also quantitative vairables with categorical measures. 

We will identify the binary measured features, and seperate them from the quantitative features. We will then normalize the quantitative columns, and remerge the two types of features to get our final independant vairables.

In [ ]:
true_binary_cols = [col for col in df if np.isin(df[col].unique(), [0, 1]).all()] 
true_binary_cols[0:10]

### Address Outliers

In [ ]:
clf = IsolationForest( behaviour = 'new', max_samples=100, random_state = 1, contamination= 'auto')
df['preds'] = clf.fit_predict(df.iloc[:,0:1])
df = df[df['preds'] == 1] #these are our outcomes that are not outliers
df = df.drop(columns = ['preds'])

In [ ]:
X = df.iloc[:,1:548]
Y = df.iloc[:,0:1]

## 2. Model Developement

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split (X, Y, test_size=0.3, random_state=0)

### Feature Selection
We will utalise an OLS Backwards Elimination method to only compose the model of statistically significant product features (independent vairables) on the sales outcome (dependent vairable). Thereafter, we will check for multi-collinear vairables and eliminate any discrepancies.

In [ ]:
y = y_train
cols = list(x_train.columns)
pmax = 1
while (len(cols)>0):
    '''This automates the backwards elimination OLS method'''
    p = []
    X_1 = x_train[cols]
    X_1 = sm.add_constant(X_1)
    model = sm.OLS(y,X_1).fit()
    p = pd.Series(model.pvalues.values[0:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print("Product features that have statistical significance on the outcome vairable: " + str(selected_features_BE))

In [ ]:
x_train = x_train.filter(items=selected_features_BE, axis=1) #our significant independant vairables only

### Check for Multi-Coliniarity

In [ ]:
# Compute the correlation matrix
corr = x_train.corr()

# Generate heatmap
sns.heatmap(corr, center=0)

On the forefront from the above heat map, we can see that some features are correlated with each other.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
[variance_inflation_factor(x_train.values, j) for j in range(x_train.shape[1])]

def calculate_vif(x):
    thresh = 10.0
    output = pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values, j) for j in range(x.shape[1])]
    for i in range(1,k):
        a = np.argmax(vif)
        if vif[a] <= thresh :
            break
        if i == 1 :          
            output = x.drop(x.columns[a], axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
        elif i > 1 :
            output = output.drop(output.columns[a],axis = 1)
            vif = [variance_inflation_factor(output.values, j) for j in range(output.shape[1])]
    return(output)

selected_features = calculate_vif(x_train)

In [ ]:
x_train = selected_features
x_train.head()

In [ ]:
x_test = x_test.filter(items=x_train.columns, axis=1)

## 3. Test Model

In [ ]:
regr = linear_model.LinearRegression()
x = np.asanyarray(x_train)
y = np.asanyarray(y_train)
regr.fit (x, y)

In [ ]:
y_hat= regr.predict(x_test)
y_hat = np.where(y_hat<0, 0, y_hat)
y_hat = np.around(y_hat, decimals=0)
x = np.asanyarray(x_test)
y = np.asanyarray(y_test)
print("Residual sum of squares: %.2f"
% np.mean((y_hat - y) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(x, y))

A final accuracy of 95% is reported from our model (0.95 variance score). Let's see the results visualised below.

In [ ]:
def DistributionPlot(y, yhat):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    ax1 = sns.distplot(y, hist=True, color="r", label='Actual')
    ax2 = sns.distplot(yhat, hist=True, color="b", label='Predicted', ax=ax1)
    plt.title('Distribution Plot of Predicted Sales vs Actual Sales', size=16)
    plt.legend()
    plt.show()

DistributionPlot(y, y_hat)